# 20240805 - Liver transplant analysis

This notebook has the code necessary to reproduce the couple of data findings in the South Dakota News Watch article, ["Alcohol's impact: 35, dry and dying while waiting for a transplant,"](https://www.sdnewswatch.org/chronic-liver-disease-cirrhosis-killing-south-dakota-30-year-olds/) published Aug. 5, 2024.

This article includes findings from analyzing portions of the [UNOS STAR file](https://optn.transplant.hrsa.gov/data/view-data-reports/request-data/). See [README.md](README.md) for more details on the data.

- [Setup and summary stats](#Setup-and-summary-stats)
    - [Read in liver file code lookup](#Read-in-liver-file-code-lookup)
    - [Read in STAR file data dictionary](#Read-in-STAR-file-data-dictionary)
    - [Read in main liver data file](#Read-in-main-liver-data-file)
    - [Flag transplants](#Flag-transplants)
    - [Flag alcohol diagnoses](#Flag-alcohol-diagnoses)
- [Findings](#Findings)

## Setup and summary stats

In [55]:
import math
from datetime import date

from utils import get_df

import pandas as pd

In [56]:
pd.set_option('display.max_columns', None)

### Read in liver file code lookup

We can use this to look up pieces of encoded data.

In [57]:
df_code_dict = get_df(
    'docs/CODE DICTIONARY - FORMATS 202406/Liver/LIVER_FORMATS_FLATFILE.DAT',
    encoding='cp1252'
)

In [58]:
df_code_dict.head()

,LABEL,FMTNAME,TYPE,CODE
0,O,ABO,C,1
1,A,ABO,C,2
2,B,ABO,C,3
3,AB,ABO,C,4
4,A1,ABO,C,5


### Read in STAR file data dictionary

Mainly to check the variable start/end dates.

In [59]:
df_star_data_dict = pd.read_excel(
    'docs/STAR User Guide/STAR File Data Dictionary.xlsx',
    sheet_name='LIVER_DATA',
    skiprows=1
)

In [60]:
df_star_data_dict.head()

,VARIABLE NAME,DESCRIPTION,FORM,VAR START DATE,VAR END DATE,FORM SECTION,DATA TYPE,SAS ANALYSIS FORMAT,COMMENT
0,ABO,RECIPIENT BLOOD GROUP @ REGISTRATION,TCR,1987-10-01,NaT,CLINICAL INFORMATION,CHAR(3),ABO,NaN
1,ABO_DON,DONOR BLOOD TYPE,DDR/LDR,1987-10-01,NaT,DONOR INFORMATION,CHAR(3),ABO,NaN
2,ABO_MAT,DONOR-RECIPIENT ABO MATCH LEVEL,CALCULATED,NaT,NaT,NaN,CHAR(1),ABOMAT,NaN
3,ACADEMIC_LEVEL_TCR,ACADEMIC ACTIVITY LEVEL AT LISTING,TCR,2004-06-30,NaT,CANDIDATE INFORMATION,NUM,ACADLVLKI,NaN
4,ACADEMIC_LEVEL_TRR,ACADEMIC ACTIVITY LEVEL AT TRANSPLANT,TRR,2004-06-30,NaT,PATIENT STATUS,NUM,ACADLVLKI,NaN


### Read in main liver data file

In [61]:
# load the LIVER_DATA.DAT file
df = get_df(
    'data/liver/LIVER_DATA.DAT',
    parse_dates=[
        'PX_STAT_DATE',
        'INIT_DATE'
    ]
)

In [62]:
# how many records are missing a list year?
len(df[df['LISTYR'].isnull()])

1167

In [63]:
# We're only going to use data where the list year is >= 1994
df = df[df['LISTYR'] >= 1994]

In [64]:
# how many records total?
total_records = len(df)
print(f'{total_records:,} events')

343,836 events


In [65]:
# any nulls in patient ID column?
print(len(df[df['PT_CODE'].isnull()]), 'nulls')

0 nulls


In [66]:
# what are max and min dates (formatted)?
date_first = df['INIT_DATE'].min().strftime("%B %Y")
date_last = df['INIT_DATE'].max().strftime("%B %Y")

# there are a few records past June 30, 2024, but we're going to
# stick with the documentation and cap the data at the end of 6/24
date_last = 'June 2024'

print(f'Earliest event: {date_first}')
print(f'Most recent event: {date_last}')

Earliest event: January 1994
Most recent event: June 2024


In [67]:
# how many unique candidates?
# get the count of unique patient codes
total_candidates_unique_count = df['PT_CODE'].nunique()
print(f'{total_candidates_unique_count:,} candidates')

300,687 candidates


In [68]:
# how many unique donors?
# get the count of unique donor IDs
total_donors_unique_count = df['DONOR_ID'].nunique()
print(f'{total_donors_unique_count:,} donors')

197,292 donors


### Flag transplants

In [69]:
# how many candidates got a transplant? for this,
# we want to make sure that every record associated with a candidate
# shows whether they have (at some point) gotten a transplant

# step one: get a unique list of all PT_CODE values that are
# associated with a transplant record -- i.e., the TRR_ID_CODE
# value is not null
pt_codes_w_tx = df[df['TRR_ID_CODE'].notnull()]['PT_CODE'].unique()

# set a convenience flag on _every_ event record showing whether this
# candidate has ever been associated with a tx record
df['has_tx'] = df['PT_CODE'].isin(pt_codes_w_tx)

In [70]:
# checks out
df[df['has_tx']]['PT_CODE'].nunique() == df[df['TRR_ID_CODE'].notnull()]['PT_CODE'].nunique()

True

### Flag alcohol diagnoses

Candidates who were listed with a primary or secondary diagnosis associated with an alcohol-related disease will have one of these codes present in the `DGN_TCR`, `DGN2_TCR` or `DIAG` fields, according to UNOS:
- 4215 – Alcoholic Cirrhosis
- 4216 – Alcoholic Cirrhosis with Hepatitis C
- 4217 – Acute Alcoholic Hepatitis
- 4218 – Acute Alcohol-Associated Hepatitis with or without Cirrhosis
- 4219 – Alcohol-Associated Cirrhosis without Acute Alcohol-Associated Hepatitis

Here, we're going to check for the presence of these codes and set a flag on each record.

In [71]:
# check the variable start dates for those columns
df_star_data_dict[df_star_data_dict['VARIABLE NAME'].isin(['DGN_TCR', 'DGN2_TCR', 'DIAG'])]

,VARIABLE NAME,DESCRIPTION,FORM,VAR START DATE,VAR END DATE,FORM SECTION,DATA TYPE,SAS ANALYSIS FORMAT,COMMENT
87,DGN_TCR,PRIMARY DIAGNOSIS AT TIME OF LISTING,TCR,1994-04-01,NaT,CLINICAL INFORMATION,NUM,ALL_DGN,NaN
88,DGN2_TCR,SECONDARY DIAGNOSIS AT TIME OF LISTING,TCR,1994-04-01,NaT,CLINICAL INFORMATION,NUM,ALL_DGN,NaN
92,DIAG,RECIPIENT PRIMARY DIAGNOSIS,TRR>TCR,1987-10-01,NaT,PATIENT STATUS/CLINICAL INFORMATION,NUM,ALL_DGN,Primary diagnosis was not collected on TCR unt...


In [72]:
# check - see which code labels include the word 'alcohol'
df_code_dict[df_code_dict['LABEL'].fillna('').str.contains('alcohol', case=False)]

,LABEL,FMTNAME,TYPE,CODE
2582,ALCOHOLIC CIRRHOSIS,LI_DGN,N,4215
2583,ALCOHOLIC CIRRHOSIS WITH HEPATITIS C,LI_DGN,N,4216
2584,ACUTE ALCOHOLIC HEPATITIS,LI_DGN,N,4217
2585,ACUTE ALCOHOL-ASSOCIATED HEPATITIS WITH OR WIT...,LI_DGN,N,4218
2586,ALCOHOL-ASSOCIATED CIRRHOSIS WITHOUT ACUTE ALC...,LI_DGN,N,4219


In [73]:
alcohol_codes = set([int(x) for x in df_code_dict[df_code_dict['LABEL'].fillna('').str.contains('alcohol', case=False)]['CODE']])

In [74]:
alcohol_codes

{4215, 4216, 4217, 4218, 4219}

In [75]:
def has_alcohol_diagnosis(row):
    ''' function to check a row of liver data to see if any of the alcohol codes are present '''
    diag_codes = [
        row['DGN_TCR'],
        row['DGN2_TCR'],
        row['DIAG']
    ]

    if set(diag_codes).intersection(alcohol_codes):
        return True

    return False

In [76]:
df['alcohol_diagnosis'] = df.apply(
    has_alcohol_diagnosis,
    axis=1
)

In [77]:
# to guard against the possibility of a candidate record not reflecting the alcohol diagnostic
# code for that candidate, set the 'alcohol_diagnosis' flag on ALL records for candidates who had an
# alcohol diagnosis in ANY of their records

# (belt and suspenders -- our numbers didn't change after we tested against this approach)

pt_codes_w_alc = df[df['alcohol_diagnosis']]['PT_CODE'].unique()

# now set a convenience flag on each event record -- has this patient
# ever been associated with an alcohol diagnosis record?
df['alcohol_diagnosis'] = df['PT_CODE'].isin(pt_codes_w_alc)

In [78]:
# grab a unique list of candidates for use later - get latest record based on
# `PX_START_DATE`
df_uniq_cands = df.sort_values('PX_STAT_DATE', ascending=False).drop_duplicates(
    subset='PT_CODE',
    keep='first'
)

## Findings

### "Most patients who need a liver aren't this young: Last year, the average age of a person listed for a potential transplant was 56."

Taylor is 35. Summary stats for `INIT_AGE` for candidates and `AGE_DON` values across the U.S. in this time period:
- Candidate age mean: 50.33
- Candidate age median: 54.00

Last year (2023), the median age of a listed candidate was 56.

In [79]:
# how many records are missing `INIT_AGE`?
len(df[df['INIT_AGE'].isnull()])

0

In [80]:
# max/min for `INIT_AGE` (candidate age at listing)
# and `AGE_DON` (donor age)

print('Age range:')
print('- Candidates:', df['INIT_AGE'].min(), '-', df['INIT_AGE'].max())
print('- Donors:', df['AGE_DON'].min(), '-', df['AGE_DON'].max())

Age range:
- Candidates: 0.0 - 86.0
- Donors: 0.0 - 98.0


In [81]:
# grab mean/median age for candidates and donors
age_mean_candidates_us = df_uniq_cands['INIT_AGE'].mean()
age_median_candidates_us = df_uniq_cands['INIT_AGE'].median()

age_str = '\n  - '.join([
    f'Candidate age mean: {age_mean_candidates_us:.2f}',
    f'Candidate age median: {age_median_candidates_us:.2f}',
])

# FINDING
print(f'Nationally:\n  - {age_str}')

Nationally:
  - Candidate age mean: 50.33
  - Candidate age median: 54.00


In [82]:
# look at mean/median candidate INIT_AGE broken out by listing year
df_uniq_cands[['LISTYR', 'INIT_AGE']].groupby('LISTYR').agg(['median', 'mean'])

INIT_AGE           
         median       mean
LISTYR                    
1994.0     46.0  42.597746
1995.0     48.0  44.759796
1996.0     48.0  45.222172
1997.0     48.0  45.266910
1998.0     49.0  46.174994
1999.0     50.0  46.909588
2000.0     50.0  46.907593
2001.0     50.0  47.025358
2002.0     51.0  47.568560
2003.0     51.0  48.264923
2004.0     52.0  48.744426
2005.0     52.0  48.817034
2006.0     53.0  49.220638
2007.0     53.0  49.564003
2008.0     54.0  50.043264
2009.0     55.0  50.588392
2010.0     55.0  51.274637
2011.0     56.0  51.916675
2012.0     56.0  52.269907
2013.0     57.0  52.488136
2014.0     57.0  52.861046
2015.0     57.0  52.551176
2016.0     57.0  52.781807
2017.0     57.0  52.754816
2018.0     57.0  52.960743
2019.0     57.0  53.187066
2020.0     57.0  52.754462
2021.0     56.0  52.155509
2022.0     56.0  51.955223
2023.0     56.0  52.072490
2024.0     56.0  52.025466

In [83]:
# break out tx rate by LISTYR

df_listyr_x_tx = pd.pivot_table(
    df_uniq_cands[['LISTYR', 'has_tx']],
    index='LISTYR',
    columns='has_tx',
    aggfunc=len
).reset_index()

In [84]:
df_listyr_x_tx.columns = ['LISTYR', 'tx_no', 'tx_yes']

In [86]:
df_listyr_x_tx['tx_pct'] = (df_listyr_x_tx['tx_yes'] / (df_listyr_x_tx['tx_yes'] + df_listyr_x_tx['tx_no'])) * 100

In [87]:
df_listyr_x_tx

,LISTYR,tx_no,tx_yes,tx_pct
0,1994.0,1687,3459,67.217256
1,1995.0,2141,3933,64.751399
2,1996.0,2525,4150,62.172285
3,1997.0,2923,4203,58.981196
4,1998.0,3417,4469,56.670048
5,1999.0,4249,4533,51.616944
6,2000.0,4331,4532,51.133928
7,2001.0,4257,4616,52.022991
8,2002.0,3321,4497,57.521105
9,2003.0,3364,5163,60.548845


### "Over the years, roughly two out of three candidates ultimately received a liver."

In [88]:
df_cand_w_tx = df[df['has_tx']]
total_candidates_w_tx = df_cand_w_tx['PT_CODE'].nunique()

# and get the % of total
tx_pct = total_candidates_w_tx / total_candidates_unique_count

print(f'In the U.S., from {date_first} to {date_last}, {total_candidates_w_tx:,} of {total_candidates_unique_count:,} liver transplant candidates received a transplant ({tx_pct:.2%}).')
print('Of course, this is just a snapshot in time: Many people who were listed recently will ultimately get a transplant.')

In the U.S., from January 1994 to June 2024, 188,056 of 300,687 liver transplant candidates received a transplant (62.54%).
Of course, this is just a snapshot in time: Many people who were listed recently will ultimately get a transplant.


### "Thirty years ago, about a quarter of the candidates were listed with an alcohol-related liver disease. Last year, they represented nearly half."

In [89]:
# how many candidates were listed with an alcohol diagnosis?
df_alcohol_diag = df[df['alcohol_diagnosis']]
alcohol_cand_total = df_alcohol_diag['PT_CODE'].nunique()

In [90]:
alcohol_cand_pct = alcohol_cand_total / total_candidates_unique_count

cands_alcohol_got_tx_count = df_alcohol_diag[df_alcohol_diag['has_tx']]['PT_CODE'].nunique()
cands_alcohol_got_tx_pct = cands_alcohol_got_tx_count / alcohol_cand_total

print(f'Of {total_candidates_unique_count:,} liver donation candidates across the U.S., {alcohol_cand_total:,} were listed with a primary or secondary diagnosis of an alcohol-related disease ({alcohol_cand_pct:.2%}).')
print(f'Of those {alcohol_cand_total:,} candidates who were listed with an alcohol diagnosis, {cands_alcohol_got_tx_count:,} received a transplant ({cands_alcohol_got_tx_pct:.2%}).')

Of 300,687 liver donation candidates across the U.S., 91,708 were listed with a primary or secondary diagnosis of an alcohol-related disease (30.50%).
Of those 91,708 candidates who were listed with an alcohol diagnosis, 55,895 received a transplant (60.95%).


In [91]:
# note: vetted against manually filtered data as well

# show unique candidates, LISTYR x alcohol diagnosis
df_alc_by_listyr_uniq =  pd.pivot_table(
    df_uniq_cands[['LISTYR', 'alcohol_diagnosis']],
    index='LISTYR',
    columns='alcohol_diagnosis',
    aggfunc=len
).reset_index()

In [92]:
df_alc_by_listyr_uniq.head()

alcohol_diagnosis,LISTYR,False,True
0,1994.0,3920,1226
1,1995.0,4424,1650
2,1996.0,4798,1877
3,1997.0,5223,1903
4,1998.0,5620,2266


In [93]:
df_alc_by_listyr_uniq.columns = ['LISTYR', 'alcohol_diag_no', 'alcohol_diag_yes']

In [94]:
df_alc_by_listyr_uniq.head()

,LISTYR,alcohol_diag_no,alcohol_diag_yes
0,1994.0,3920,1226
1,1995.0,4424,1650
2,1996.0,4798,1877
3,1997.0,5223,1903
4,1998.0,5620,2266


In [95]:
df_alc_by_listyr_uniq['alcohol_diag_pct'] = (df_alc_by_listyr_uniq['alcohol_diag_yes'] / (df_alc_by_listyr_uniq['alcohol_diag_yes'] + df_alc_by_listyr_uniq['alcohol_diag_no'])) * 100

In [96]:
df_alc_by_listyr_uniq

,LISTYR,alcohol_diag_no,alcohol_diag_yes,alcohol_diag_pct
0,1994.0,3920,1226,23.824330
1,1995.0,4424,1650,27.164965
2,1996.0,4798,1877,28.119850
3,1997.0,5223,1903,26.705024
4,1998.0,5620,2266,28.734466
5,1999.0,6375,2407,27.408335
6,2000.0,6449,2414,27.236827
7,2001.0,6557,2316,26.101657
8,2002.0,5860,1958,25.044768
9,2003.0,6437,2090,24.510379


In [97]:
df_alc_by_listyr_uniq.to_clipboard(index=False)

In [98]:
# spot-checking 2023 numbers from source
test_23 = df_uniq_cands[df_uniq_cands['LISTYR'] == 2023]
test_23_has_alc = test_23[test_23['alcohol_diagnosis']]
test_23_no_alc = test_23[~test_23['alcohol_diagnosis']]
print(len(test_23_has_alc))
print(len(test_23_no_alc))

6142
7584


### "The majority of these patients [with an alcohol-related liver disease] ended up with a liver."

In [99]:
count_alc = df_alcohol_diag['PT_CODE'].nunique()
count_alc_tx_yes = df_alcohol_diag[df_alcohol_diag['has_tx']]['PT_CODE'].nunique()

alc_tx_yes_pct = count_alc_tx_yes / count_alc

print(f'Of {count_alc:,} candidates with an alcohol-related diagnosis, {count_alc_tx_yes:,} ultimately got a transplant ({alc_tx_yes_pct:.2%}).')

Of 91,708 candidates with an alcohol-related diagnosis, 55,895 ultimately got a transplant (60.95%).


In [100]:
# break this out by LISTYR
df_uniq_cands_alc = df_uniq_cands[df_uniq_cands['alcohol_diagnosis']]

In [101]:
df_uniq_cands_alc_tx_x_listyr = pd.pivot_table(
    df_uniq_cands_alc[['LISTYR', 'has_tx']],
    index='LISTYR',
    columns='has_tx',
    aggfunc=len
).reset_index()

df_uniq_cands_alc_tx_x_listyr.columns = ['LISTYR', 'tx_no', 'tx_yes']

In [102]:
df_uniq_cands_alc_tx_x_listyr.head()

,LISTYR,tx_no,tx_yes
0,1994.0,377,849
1,1995.0,609,1041
2,1996.0,739,1138
3,1997.0,874,1029
4,1998.0,1056,1210


In [103]:
df_uniq_cands_alc_tx_x_listyr['tx_pct'] = (df_uniq_cands_alc_tx_x_listyr['tx_yes'] / (df_uniq_cands_alc_tx_x_listyr['tx_yes'] + df_uniq_cands_alc_tx_x_listyr['tx_no'])) * 100

In [104]:
df_uniq_cands_alc_tx_x_listyr

,LISTYR,tx_no,tx_yes,tx_pct
0,1994.0,377,849,69.249592
1,1995.0,609,1041,63.090909
2,1996.0,739,1138,60.628663
3,1997.0,874,1029,54.072517
4,1998.0,1056,1210,53.398058
5,1999.0,1236,1171,48.649771
6,2000.0,1273,1141,47.265949
7,2001.0,1182,1134,48.963731
8,2002.0,878,1080,55.158325
9,2003.0,902,1188,56.842105
